In [16]:
# The code was removed by Watson Studio for sharing.

# Coursera Capstone Project

**Jacob Bills**  
Data Systems Engineer

# Objective <a name='objective' />
<p>To determine the best type of business to open based on data among the most popular venues between two of the largest cities in West Virginia</p>

# Background <a name='background' />
<p>Huntington and Charelston WV are the two most populated cities in West Virginia <a href='http://www.togetherweteach.com/TWTIC/uscityinfo/48wv/wvpopr/48wvpr.htm'>(source)</a>. I will use a publicy available list of zip codes in WV to lookup location data using the Google Cloud Platforms Geocoder API. Then I will use Foursquare API to lookup popular venues for each location. I will use this data to create a geographical clustering model. Using this model I will build a recommender system so you may determine which city is the best for opening your business</p>

# In this document
* <a href="#objective">Objective</a>
* <a href="#background">Background</a>
* <a href="#datacollection">Data Collection</a>
  * <a href="#datacollectionzipcode">WV zip code data</a>
  * <a href="#datacollectionlocation">Huntington and Charleston location data</a>
  * <a href="#datacollectionvenue">Huntington and Charleston venue data</a>
* <a href="#datasummary">Summary of Data Collection</a>
  * <a href="#datasummaryviz">Data visualization</a>
  * <a href="#datasummaryhuntington">Most popular Huntington venues</a>
  * <a href="#datasummarycharleston">Most popular Charleston venues</a>
* <a href="#datamodeling">Data Modeling (TODO Week 5)</a>

# Data Collection <a name='datacollection' />

### Get zip code data for West Virginia <a name='datacollectionzipcode' />
<p>I found this <a href='https://www.zipcodestogo.com/West%20Virginia/'>web site</a> after googling for West Virginia zip codes. I transformed the table and dropped the Zip Code Map column, added latitude column, and added longitude column</p>

In [15]:
import pandas as pd
import numpy as np
wvdata = pd.read_html('https://www.zipcodestogo.com/West%20Virginia/')
wvdata[1].iloc[1:].iloc[0]
headers= wvdata[1].iloc[1:].iloc[0]
wvdata = pd.DataFrame(wvdata[1].iloc[2:].values, columns=headers)
wvdata.columns.name =''
wvdata.drop(['Zip Code Map'],axis=1,inplace=True)
wvdata.insert(3,'Latitude',np.nan)
wvdata.insert(4,'Longitude',np.nan)
unique_zipcodes=pd.DataFrame(wvdata['Zip Code'].value_counts())
print("there are ", unique_zipcodes.shape[0], " zipcodes")
wvdata.head(5)

there are  861  zipcodes


,Zip Code,City,County,Latitude,Longitude
0,24701,Bluefield,Mercer,NaN,NaN
1,24712,Athens,Mercer,NaN,NaN
2,24714,Beeson,Mercer,NaN,NaN
3,24715,Bramwell,Mercer,NaN,NaN
4,24716,Bud,Wyoming,NaN,NaN


<p>
For a sandbox tier Foursquare account like mine, <a href='https://foursquare.com/developers/account/billing'>here</a> are the prescribed rate limits for each type of API call against their platform
    
* 950 Regular Calls/Day
* 50 Premium Calls/Day
* 1 Photo per Venue
* 1 Tip per Venue

Since there are 861 unique zip codes I'll limit the result set to the cities of Huntington and Charleston WV, otherwise I will quickly reach the daily limit.
</p>

In [18]:
twocities=wvdata.loc[(wvdata['City'] == 'Huntington') | (wvdata['City'] == 'Charleston')]
print("there are ",twocities.shape[0], " zipcodes among the Huntington and Charelston WV areas.",twocities['Zip Code'].value_counts().shape[0]," of which are unique")

there are  87  zipcodes among the Huntington and Charelston WV areas. 87  of which are unique


<p>So I reduced it down to only 87 zip codes which gives plenty of room before hitting the daily rate limit for the Foursquare API.</p>

### Get location data for the selected zip codes using Google Cloud Platform Geocoder API. <a name='datacollectionlocation' />
*this requires you setup an api key for geocoder api and store it in the google_api_key variable*

<p>For each zip code I used the <a href='https://developers.google.com/maps/documentation/geocoding/'>Google Cloud Platforms Geocoder API</a> to gather the latitude and longitude data. You can find information about the pricing model for this API <a href='https://developers.google.com/maps/documentation/geocoding/usage-and-billing'>here</a>. If you already have a Google Cloud Platform account you can find the exact pricing <a href='https://console.cloud.google.com/apis/library/geocoding-backend.googleapis.com'>here</a>. I will store the location data to the Watson Studio project in order to limit the frequency of calls against this API.
</p>

In [179]:
import requests
for index, row in twocities.iterrows():
    addr=row['City'] + ' WV, ' + row['Zip Code']
    url="https://maps.googleapis.com/maps/api/geocode/json"
    p=[('address',addr), ('key',google_api_key)]
    loc=requests.get(url,params=p).json()['results'][0]['geometry']['location']
    twocities.at[index, 'Latitude'] = loc['lat']
    twocities.at[index, 'Longitude'] = loc['lng']

twocities.head()

,Zip Code,City,County,Latitude,Longitude
259,25301,Charleston,Kanawha,38.351711,-81.633647
260,25302,Charleston,Kanawha,38.388374,-81.598025
261,25303,Charleston,Kanawha,38.363269,-81.685681
262,25304,Charleston,Kanawha,38.279687,-81.614468
263,25305,Charleston,Kanawha,38.337970,-81.610700


*Here is the code that saves the location data results to the Watson Studio project for later retreivel.*

In [29]:
# save raw location data to project
project.save_data(data=twocities.to_csv(index=False),file_name='twocities.csv',overwrite=True)

<p>Some zip codes may have identical longitude,latitude. I perform a group by operation to further limit the result set to only unique longitude,latitude values</p>

In [19]:
#load results from gcp api csv file
my_file = project.get_file("twocities.csv")
my_file.seek(0)
df=pd.read_csv(my_file)
df["Zip Code"]= df["Zip Code"].astype(str)

#without loading results from file
#df=twocities

latlon=df.groupby(['Latitude','Longitude','City','County'])['Zip Code'].agg([('count', 'count'), ('Zip Code', ','.join)]).reset_index()
latlon.sort_values(by=['count'], ascending=False).head(10)

,Latitude,Longitude,City,County,count,Zip Code
13,38.350000,-81.630000,Charleston,Kanawha,22,"25321,25322,25323,25324,25325,25326,25327,2532..."
29,38.420000,-82.440000,Huntington,Cabell,18,"25706,25708,25712,25713,25714,25716,25717,2571..."
27,38.420000,-82.470000,Huntington,Cabell,10,"25770,25771,25772,25773,25774,25775,25776,2577..."
28,38.420000,-82.450000,Huntington,Cabell,5,"25707,25710,25711,25715,25723"
26,38.416135,-81.767764,Charleston,Kanawha,1,25313
21,38.380792,-82.486105,Huntington,Wayne,1,25704
22,38.383407,-81.674729,Charleston,Kanawha,1,25387
23,38.388374,-81.598025,Charleston,Kanawha,1,25302
24,38.409359,-82.361463,Huntington,Cabell,1,25705
25,38.410000,-82.460000,Huntington,Cabell,1,25709


In [20]:
print("there are ", latlon.shape[0], " unique latitude,longitude pairs across the 87 zip codes")

there are  36  unique latitude,longitude pairs across the 87 zip codes


### Get venue recommendations from the Foursquare API using the latitude and longitude data. <a name='datacollectionvenue' />
<p>Geocoder API returned the same location data up to 22 times for different zip codes. This significantly reduces the amount of API calls needed to be performed for gathering venue data from the Foursquare API.</p>
<p><a href='https://developer.foursquare.com/docs/api/venues/explore'>This</a> is the endpoint that I will use for retreiving venue recomendations using the latitude,longitude data. It is not a premium endpoint so I can hit this up to 950 times a day</p>

In [21]:
LIMIT = 100
radius = 500
def getNearbyVenues(label, names, latitudes, longitudes, city, county, radius=500):
    
    venues_list=[]
    for name, lat, lng, cty, cy in zip(names, latitudes, longitudes, city, county):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            cty,
            cy,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [label, 
                  label + ' Latitude', 
                  label + ' Longitude', 
                  'City',
                  'County',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [107]:
dt_venues = getNearbyVenues(label='Zip Code',names=latlon['Zip Code'],
                                   latitudes=latlon['Latitude'],
                                   longitudes=latlon['Longitude'],
                                   city=latlon['City'],county=latlon['County'])

25315
25365
25304
25314
25309
25306
25364
25317
25396
25305
25361
25312
25389
25321,25322,25323,25324,25325,25326,25327,25328,25329,25330,25331,25332,25333,25334,25335,25336,25337,25338,25339,25350,25358,25375
25392
25301
25362
25311
25303
25701
25357
25704
25387
25302
25705
25709
25313
25770,25771,25772,25773,25774,25775,25776,25777,25778,25779
25707,25710,25711,25715,25723
25706,25708,25712,25713,25714,25716,25717,25718,25719,25720,25721,25722,25724,25725,25726,25727,25728,25729
25755
25703
25702
25356
25360
25320


In [130]:
print("there are ", dt_venues.shape[0], " venues in the result set")

there are  474  venues in the result set


In [109]:
# save raw location data to project
project.save_data(data=dt_venues.to_csv(index=False),file_name='wv_huntington_charleston_venues.csv',overwrite=True)

{'file_name': 'wv_huntington_charleston_venues.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'capstoneproject-donotdelete-pr-enazjz6glyx6ja',
 'asset_id': 'ddfaae23-c283-4e21-9847-9d97451876af'}

In [22]:
import pandas as pd
#load results from venue csv
my_file2 = project.get_file("wv_huntington_charleston_venues.csv")
my_file2.seek(0)
dt_venues=pd.read_csv(my_file2)

#without loading results from file
dt_unique_venues=dt_venues
dt_unique_venues=dt_unique_venues.groupby(['Venue','Venue Latitude','Venue Longitude','Venue Category','City','County'])['Zip Code'].agg([('count', 'count'), ('Zip Code', ','.join)]).reset_index()
huntington_venues=dt_unique_venues[dt_unique_venues['City']=='Huntington']
charleston_venues=dt_unique_venues[dt_unique_venues['City']=='Charleston']
huntington_venues=huntington_venues.replace(to_replace=r'\'', value="", regex=True)
charleston_venues=charleston_venues.replace(to_replace=r'\'', value="", regex=True)
print("Huntington:",huntington_venues.shape[0],"total venues and",huntington_venues['Venue Category'].value_counts().shape[0], "categories")
print("Charleston:",charleston_venues.shape[0],"total venues and",charleston_venues['Venue Category'].value_counts().shape[0], "categories")
print("Total:",dt_unique_venues.shape[0], "total venues and",dt_unique_venues['Venue Category'].value_counts().shape[0], "categories")

Huntington: 124 total venues and 79 categories
Charleston: 160 total venues and 87 categories
Total: 284 total venues and 124 categories


In [29]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

hlat=(huntington_venues['Venue Latitude'].max()+huntington_venues['Venue Latitude'].min())/2
hlon=(huntington_venues['Venue Longitude'].max()+huntington_venues['Venue Longitude'].min())/2
clat=(charleston_venues['Venue Latitude'].max()+charleston_venues['Venue Latitude'].min())/2
clon=(charleston_venues['Venue Longitude'].max()+charleston_venues['Venue Longitude'].min())/2
map_wv = [
    {'map':folium.Map(location=[hlat, hlon], zoom_start=12,width='100%'),'venues':huntington_venues,'most_popular':None},
    {'map':folium.Map(location=[clat, clon], zoom_start=12,width='100%'),'venues':charleston_venues,'most_popular':None}
]

# add markers to map
for m in map_wv:
  for lat, lng, borough, neighborhood in zip(m['venues']['Venue Latitude'], m['venues']['Venue Longitude'], m['venues']['Venue Category'], m['venues']['Venue']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(m['map'])  

# Summary of Data Collection <a name='datasummary' />

### Plot venue locations for the two cities side by side Huntington on the left and Charleston on the right <a name='datasummaryviz' />

In [30]:
from IPython.core.display import display, HTML

htmlmap = HTML('<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 50%; margin: 0 auto; border: 2px solid black"></iframe>'
               '<iframe srcdoc="{}" style="float:right; width: {}px; height: {}px; display:inline-block; width: 50%; margin: 0 auto; border: 2px solid black"></iframe>'
               .format(map_wv[0]['map'].get_root().render().replace('"', '&quot;'),500,500,
                       map_wv[1]['map'].get_root().render().replace('"', '&quot;'),500,500))
display(htmlmap)

In [31]:
import numpy as np
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Zip Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
for m in map_wv:
    # one hot encoding
    dt_onehot = pd.get_dummies(m['venues'][['Venue Category']], prefix="", prefix_sep="")
    dt_onehot['Zip Code'] = m['venues']['Zip Code'] 
    fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
    dt_onehot = dt_onehot[fixed_columns]
    dt_grouped = dt_onehot.groupby('Zip Code').mean().reset_index()
    dt_grouped.head()

    # create a new dataframe
    m['most_popular'] = pd.DataFrame(columns=columns)
    m['most_popular']['Zip Code'] = dt_grouped['Zip Code']

    for ind in np.arange(dt_grouped.shape[0]):
        m['most_popular'].iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)

### Most popular Huntington venues <a name='datasummaryhuntington' />

In [32]:
map_wv[0]['most_popular'].head()

,Zip Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,25703,Fast Food Restaurant,Bar,Mexican Restaurant,Rental Car Location,Football Stadium,Pizza Place,Seafood Restaurant,Automotive Shop,Gas Station,Donut Shop
1,25705,Gas Station,Discount Store,Comfort Food Restaurant,Chinese Restaurant,Middle Eastern Restaurant,Breakfast Spot,Bowling Alley,Mexican Restaurant,Donut Shop,Grocery Store
2,"25706,25708,25712,25713,25714,25716,25717,2571...",American Restaurant,Bar,Seafood Restaurant,Italian Restaurant,Pub,Coffee Shop,Mexican Restaurant,Clothing Store,Mobile Phone Shop,Bank
3,"25707,25710,25711,25715,25723",Gas Station,American Restaurant,Fast Food Restaurant,BBQ Joint,Event Space,Indian Restaurant,Moving Target,Liquor Store,Rental Car Location,Cupcake Shop
4,"25707,25710,25711,25715,25723,25706,25708,2571...",Gastropub,Bar,Pizza Place,Japanese Restaurant,Asian Restaurant,Fast Food Restaurant,Design Studio,College Bookstore,College Rec Center,Comfort Food Restaurant


### Most popular Charleston venues <a name='datasummarycharleston' />

In [33]:
map_wv[1]['most_popular'].head()

,Zip Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,25301,Clothing Store,Gym / Fitness Center,Video Game Store,Toy / Game Store,Harbor / Marina,Kids Store,Doctors Office,Shoe Store,Fast Food Restaurant,Department Store
1,25302,Pool,Discount Store,Hotel,Cosmetics Shop,Department Store,Dessert Shop,Dive Bar,Doctors Office,Farmers Market,Fast Food Restaurant
2,25303,Hotel,Café,Bar,Fast Food Restaurant,Yoga Studio,French Restaurant,Dessert Shop,Discount Store,Dive Bar,Doctors Office
3,"25312,25301",Rock Club,Other Great Outdoors,Dive Bar,Yoga Studio,Convenience Store,Department Store,Dessert Shop,Discount Store,Doctors Office,Farmers Market
4,"25312,25321,25322,25323,25324,25325,25326,2532...",Art Museum,Gym,Gourmet Shop,Seafood Restaurant,Yoga Studio,Food Stand,Department Store,Dessert Shop,Discount Store,Dive Bar


# Data Modeling (TODO Week 5)  <a name='datamodeling' />